 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/1.%20Record%20Linkage%20between%20DBLP%20and%20Arxiv.ipynb#Running-the-record-linkage" data-toc-modified-id="Running-the-record-linkage-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Running the record linkage</a></span></li></ul></div>

This is a bash notebook that runs the code that does the simple coreference. It assumes that you have already run the notebook that [downloads and preprocesses the data](0.%20Download%20and%20Preprocess%20Data.ipynb). 

In [3]:
mkdir generated/matching

# Running the record linkage

There are a lot of different record linkage methods in the code, but in the end, it seemed to make sense to go with something simple but conservative: We say that a paper on DBLP appears on the arxiv if: (a) the title matches exactly, and (b) at least one of the author names matches. 

In [18]:
time python2 matching/match_cnf_arxiv.py \
         --arxiv-file generated/arxiv/json/arxiv_articles.json \
         --dblp-file generated/dblp/all-papers.json \
         --output-file generated/matching/all-papers-matched-titleauthor.json

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Total articles matched =  7313
Total articles checked =  82427

real	0m9.733s
user	0m8.552s
sys	0m0.890s


The below script creates a subset of: 

 * 25 matched articles
 * 25 unmatched articles whose titles and authors have Jaccard similarity > 0.5

This will then be reviewed manually to estimate the precision and recall of this poor man's matching algorithm.

In [ ]:
time python2 matching/random_subsets.py \
         --dblp-file generated/matching/all-papers-matched-titleauthor.json \
         --arxiv-file generated/arxiv/json/arxiv_articles.json \
         --threshold 0.5 --N 25 --seed 2045230 \
         --prefix generated/matching/subset_

In the above:

* "DBLP articles" is all of the articles that we extracted from DBLP
* "Matched articles" is the number of DBLP articles for which we found a match on Arxiv

(those two numbers should be the same as the previous script)

* "close articles" are those that do not have a match on arxiv, but there *is* an arxiv article whose Jaccard similarity on both titles and authors is greater than threshold
* "non-close" are the ones with no arxiv match, and no arxiv article whose Jaccard similarity on both titles and authors is greater than threshold

We should have close + non-close + matched == DBLP articles


Before doing the manual analysis, I found it helpful to write one more script that processed the data set into the exact URLs that I needed to go to online:

In [14]:
python2 matching/subsets2csv.py \
    generated/matching/subset__close.json generated/matching/subset__match.json  \
    > data/manually_labeled_coref.csv

db/conf/nips/nips2014.html#ZhuLWT14
db/conf/focs/focs2016.html#KyngS16
db/conf/icml/icml2012.html#BachrachGMG12
db/conf/nips/nips2009.html#Nowak09
db/conf/nips/nips2013.html#Zhu13
db/conf/icml/icml2015.html#TangSX15
db/conf/icml/icml2016.html#SantoroBBWL16
db/conf/nips/nips2016.html#Wei16
db/conf/iros/iros2012.html#LimLK12
db/conf/iros/iros2016.html#TamjidiCS16
Traceback (most recent call last):
  File "matching/subsets2csv.py", line 53, in <module>
    main()
  File "matching/subsets2csv.py", line 43, in main
    raise Exception("Cannot find dblp id:\n"+str(art))
Exception: Cannot find dblp id:
{u'bow_authors': [[u'hao', u'peng'], [u'shandian', u'zhe'], [u'xiao', u'zhang'], [u'yuan', u'qi']], u'title_dist': 0.7142857142857143, u'area': u'mlmining', u'url': None, u'closest_arxiv': {u'category': u'stat.ML', u'bow_authors': [[u'hao', u'peng'], [u'shandian', u'zhe'], [u'yuan', u'qi']], u'title': u'Asynchronous Distributed Variational Gaussian Processes for Regression', u'year': 2017, u'su

: 1